In [ ]:
# 1. Check GPU
import torch
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available(): print(f'GPU: {torch.cuda.get_device_name(0)}')
else: raise RuntimeError('Enable GPU!')

In [ ]:
# 2. Install packages
!pip install -q nltk gensim pillow tqdm
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

In [ ]:
# 3. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 4. Copy data
import os
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
!mkdir -p /content/artemis
!cp -r "{DRIVE_PATH}/data" /content/artemis/
!cp -r "{DRIVE_PATH}/utils" /content/artemis/
!cp -r "{DRIVE_PATH}/models" /content/artemis/
!cp "{DRIVE_PATH}/train.py" /content/artemis/
print('Copied!')

In [ ]:
# 5. Setup path
import sys
os.chdir('/content/artemis')
sys.path.insert(0, '/content/artemis')

In [ ]:
# 6. Imports
import torch, json, time, numpy as np
from utils.text_preprocessing import TextPreprocessor
from utils.data_loader import create_data_loaders
from models.cnn_lstm import create_model
from train import Trainer
print('Imports OK')

In [ ]:
# 7. Config
CONFIG = {
    'name': 'colab_cnn_glove',
    'batch_size': 32,
    'num_images': 15000,
    'epochs': 40,
    'learning_rate': 5e-5,
    'embed_dim': 300,  # GloVe uses 300
    'hidden_dim': 512,
    'attention_dim': 256,
    'dropout': 0.3,
    'embedding_type': 'glove'
}
print(f"Training: {CONFIG['name']}")
print(f"Images: ~{CONFIG['num_images']}, Epochs: {CONFIG['epochs']}")

In [ ]:
# 8. Load vocabulary and GloVe embeddings
text_proc = TextPreprocessor()
text_proc.load_vocabulary('data/processed/vocabulary.json')
print(f'Vocab: {text_proc.vocab_size}')

# Load GloVe embeddings
embedding_matrix = np.load('data/embeddings/glove_embeddings.npy')
print(f'GloVe matrix: {embedding_matrix.shape}')

In [ ]:
# 9. Data loaders
class LimitedLoader:
    def __init__(self, loader, max_batches):
        self.loader = loader
        self.max_batches = max_batches
        self.batch_size = loader.batch_size
        self.dataset = loader.dataset
    def __iter__(self):
        for i, b in enumerate(self.loader):
            if i >= self.max_batches: break
            yield b
    def __len__(self): return min(len(self.loader), self.max_batches)

loaders = create_data_loaders(text_preprocessor=text_proc, batch_size=CONFIG['batch_size'], num_workers=2, splits=['train','val'])
train_loader = LimitedLoader(loaders['train'], CONFIG['num_images']//CONFIG['batch_size'])
val_loader = LimitedLoader(loaders['val'], 50)
print(f'Train: {len(train_loader)} batches, Val: {len(val_loader)} batches')

In [ ]:
# 10. Create model with GloVe embeddings
model = create_model(
    embedding_matrix=embedding_matrix,
    vocab_size=text_proc.vocab_size,
    embed_dim=CONFIG['embed_dim'],
    decoder_dim=CONFIG['hidden_dim'],
    attention_dim=CONFIG['attention_dim'],
    dropout=CONFIG['dropout']
).to('cuda')
print(f'Params: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# 11. Train
os.makedirs(f"checkpoints/{CONFIG['name']}", exist_ok=True)
os.makedirs(f"outputs/{CONFIG['name']}", exist_ok=True)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    text_preprocessor=text_proc,
    learning_rate=CONFIG['learning_rate'],
    device='cuda',
    checkpoint_dir=f"checkpoints/{CONFIG['name']}",
    output_dir=f"outputs/{CONFIG['name']}"
)

print(f"\n{'='*70}\nSTARTING TRAINING: {CONFIG['name']}\n{'='*70}")
t0 = time.time()
history = trainer.train(num_epochs=CONFIG['epochs'])
duration = (time.time()-t0)/60

print(f"\n{'='*70}")
print(f"DONE! Duration: {duration:.1f} min")
print(f"Best BLEU: {max(history['val_bleu']):.4f}")
print(f"Best Loss: {min(history['val_loss']):.4f}")

In [ ]:
# 12. Save to Drive
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
!mkdir -p "{DRIVE_PATH}/checkpoints"
!mkdir -p "{DRIVE_PATH}/outputs"
!cp -r "checkpoints/{CONFIG['name']}" "{DRIVE_PATH}/checkpoints/"
!cp -r "outputs/{CONFIG['name']}" "{DRIVE_PATH}/outputs/"
print(f"Saved to {DRIVE_PATH}")